<a href="https://colab.research.google.com/github/AndreFRAPPIER/EatTogether/blob/main/EatTogether.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class Etudiant() :
  # fonction init stockant les informations remplis par l'étudiant (site HTML)
  def __init__(self,nom,prenom,filiere,) :
    self.NOM=nom
    self.PRENOM=prenom
    self.FILIERE=filiere

  def comparaison(self) :


etudiant1=Etudiant(#infos à mettre ici)

# BackEnd (ANCIEN Python)

In [ ]:
### COMMENTAIRE - SEANCE 05/11 ###
# ici on a le premier code Python
# qui permettra dans le code FastAPI d’utiliser des fonctions de ce programme
# par exemple si tout ce programme se nomme application
# on pourra écrire : from application import Etudiant (nom de l’objet)

from fastapi import FastAPI
from pydantic import BaseModel
from typing import List, Optional

app = FastAPI()

# ------------------ MODELES ------------------

class Etudiant:
    def __init__(self, nom, prenom, filiere, interets: List[str], disponibilites: List[str], zone: str):
        self.nom = nom
        self.prenom = prenom
        self.filiere = filiere
        self.interets = interets
        self.disponibilites = disponibilites
        self.zone = zone

    def score_match(self, autre):
        """Compare self avec un autre étudiant → retourne un score entre 0 et 100."""
        score = 0

        # Zone
        if self.zone == autre.zone:
            score += 30

        # Disponibilités communes
        dispo_communes = set(self.disponibilites) & set(autre.disponibilites)
        score += len(dispo_communes) * 10

        # Intérêts en commun
        interets_communs = set(self.interets) & set(autre.interets)
        score += len(interets_communs) * 5

        return score


class EtudiantInput(BaseModel):
    nom: str
    prenom: str
    filiere: str
    interets: List[str]
    disponibilites: List[str]
    zone: str


# ------------------ "BASE DE DONNÉES" TEMPORAIRE ------------------

etudiants: List[Etudiant] = []

# ------------------ ROUTES ------------------

@app.post("/ajouter_etudiant")
def ajouter_etudiant(data: EtudiantInput):
    """Ajoute un nouvel étudiant dans la base."""
    new_etudiant = Etudiant(**data.dict())
    etudiants.append(new_etudiant)
    return {"message": "Étudiant ajouté avec succès", "nombre_total": len(etudiants)}


@app.get("/proposer_match")
def proposer_match(nom: str):
    """Propose le meilleur match pour l'étudiant donné."""
    # Trouver l'étudiant i
    etu_i = next((etu for etu in etudiants if etu.nom == nom), None)
    if etu_i is None:
        return {"message": "Étudiant non trouvé."}

    # Comparer avec tous les autres
    scores = []
    for etu_j in etudiants:
        if etu_j is not etu_i:
            scores.append((etu_j, etu_i.score_match(etu_j)))

    # Trier du meilleur match au pire
    scores.sort(key=lambda x: x[1], reverse=True)

    if not scores:
        return {"message": "Aucun autre étudiant pour matcher."}

    best_match, best_score = scores[0]

    return {
        "match_propose": best_match.prenom + " " + best_match.nom,
        "score_de_match": best_score,
        "zone_commune": best_match.zone == etu_i.zone,
        "interets_communs": list(set(etu_i.interets) & set(best_match.interets)),
        "dispo_communes": list(set(etu_i.disponibilites) & set(best_match.disponibilites))
    }

  # Pour tester séparément nos parties de code Python:
  if __name__ == "__main__":
    etudiant1 = Etudiant(Shivani, )

  # ------------------ CSV ------------------

  from fastapi import FastAPI, Form, HTTPException
from typing import List, Dict
import csv
import os

app = FastAPI(title="EatTogether API")

CSV_FILE = "eattogether.csv"
CSV_FIELDS = ["name", "statut", "start", "end", "zone", "interests", "table"]

# -------------------------------
# 1) Fonction utilitaire pour lire le CSV
# -------------------------------
def read_csv() -> List[Dict[str, str]]:
    data = []
    if not os.path.exists(CSV_FILE):
        return data
    with open(CSV_FILE, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f, delimiter=';')
        for row in reader:
            data.append(row)
    return data

# -------------------------------
# 2) Fonction utilitaire pour écrire (réécrire) le CSV
# -------------------------------
def write_csv(data: List[Dict[str, str]]):
    with open(CSV_FILE, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=CSV_FIELDS, delimiter=';')
        writer.writeheader()
        writer.writerows(data)

# -------------------------------
# 3) Ajouter une ligne (POST)
# -------------------------------
@app.post("/inscription")
def add_entry(
    name: str = Form(...),
    statut: str = Form(...),
    start: str = Form(...),
    end: str = Form(...),
    zone: str = Form(...),
    interests: str = Form(...),
    table: str = Form(...)
):
    file_exists = os.path.isfile(CSV_FILE)

    # Si le fichier n'existe pas, créer avec l'en-tête
    with open(CSV_FILE, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f, delimiter=';')
        if not file_exists:
            writer.writerow(CSV_FIELDS)
        writer.writerow([name, statut, start, end, zone, interests, table])

    return {"status": "ok", "message": f"Ajouté {name} avec succès ✅"}

# -------------------------------
# 4)  Lire toutes les entrées (GET)
# -------------------------------
@app.get("/inscriptions")
def get_all():
    data = read_csv()
    return {"count": len(data), "data": data}

# -------------------------------
# 5) Mettre à jour une ligne (PUT)
# -------------------------------
@app.put("/update/{name}")
def update_entry(
    name: str,
    statut: str = Form(None),
    start: str = Form(None),
    end: str = Form(None),
    zone: str = Form(None),
    interests: str = Form(None),
    table: str = Form(None)
):
    data = read_csv()
    found = False

    for row in data:
        if row["name"].lower() == name.lower():
            if statut: row["statut"] = statut
            if start: row["start"] = start
            if end: row["end"] = end
            if zone: row["zone"] = zone
            if interests: row["interests"] = interests
            if table: row["table"] = table
            found = True
            break

    if not found:
        raise HTTPException(status_code=404, detail=f"Aucune entrée trouvée pour {name}")

    write_csv(data)
    return {"status": "ok", "message": f"Données de {name} mises à jour ✅"}


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List
from fastapi.middleware.cors import CORSMiddleware

# ---------------------------
# CONFIG API + CORS
# ---------------------------

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)


# ---------------------------
# MODELES
# ---------------------------

class UserInput(BaseModel):
    name : str
    statut : str
    start: str
    end: str
    zone: str | None
    interests: List[str]
    table: str | None


class Individu:
    """Un individu tel que défini par le FORMULAIRE HTML."""
    def __init__(self, data: UserInput):
        self.name = data.name
        self.statut = data.statut
        self.start = data.start
        self.end = data.end
        self.zone = data.zone
        self.interests = data.interests
        self.table = data.table

    def match_score(self, other):
        """Calcule un score de match entre 0 et 100."""

        score = 0

        # 1) Table similaire
        if self.table and other.table and self.table == other.table:
            score += 20

        # 2) Zone identique
        if self.zone and other.zone and self.zone == other.zone:
            score += 30

        # 3) Chevauchement des horaires
        if self.start <= other.end and other.start <= self.end:
            score += 20

        # 4) Intérêts communs
        commun = set(self.interests) & set(other.interests)
        score += len(commun) * 5

        return score


# ---------------------------
# BASE DE DONNÉES TEMPORAIRE
# ---------------------------

etudiants: List[Individu] = []
historique: List[Individu] = []


# ---------------------------
# ENDPOINT PRINCIPAL UTILISÉ PAR TON HTML
# ---------------------------

@app.post("/api/matches")
def trouver_match(data: UserInput):

    # Créer un nouvel étudiant
    nouvel = Individu(data)

    # Sauvegarder
    etudiants.append(nouvel)
    historique.append(nouvel)

    # Chercher des matchs
    suggestions = []

    for other in etudiants:
        if other is not nouvel:
            score = nouvel.match_score(other)

            if score > 0:  # On ne montre que si un début de match existe
                suggestions.append({
                    "name": other.name,
                    "statut": other.statut,
                    "zone": other.zone,
                    "start": other.start,
                    "end": other.end,
                    "interests": other.interests,
                    "table": other.table,
                    "matchScore": score
                })

    # Trier du meilleur match au moins bon
    suggestions.sort(key=lambda x: x["matchScore"], reverse=True)

    return {
        "matches": suggestions,
        "history": [
            {
                "name": user.name,
                "statut": user.statut,
                "table": user.table
            }
            for user in historique
        ]
    }
# def create_individu(recoit ici le dictionnaire) :
  # rentrer l'individu dans la data base

# Pour tester séparément nos parties de code Python:
if __name__ == "__main__":
  userinput1 = UserInput(
      name="Shivani",
      statut="Etudiant",
      start="12:00",
      end="12:40",
      zone="Resto U",
      interests=["Informatique", "Musique"],
      table="Etudiants"
  )

  userinput2 = UserInput(
      name="Emilie",
      statut="Etudiant",
      start="13:00",
      end="13:40",
      zone="Resto U",
      interests=["Informatique", "Musique"],
      table="Etudiants"
  )

  etudiant1 = Individu(userinput1)
  etudiant2 = Individu(userinput2)
  score = etudiant1.match_score(etudiant2)
  print(score)


60


# BackEnd (NOUVEAU Python)

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List
from fastapi.middleware.cors import CORSMiddleware
from datetime import datetime, timedelta

# ---------------------------
# CONFIG API + CORS
# ---------------------------

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)


# ---------------------------
# MODELES
# ---------------------------

class UserInput(BaseModel):
    name: str
    statut: str
    start: str
    end: str
    zone: str | None
    interests: List[str]
    table: str | None


class Individu:
    """Un individu tel que défini par le FORMULAIRE HTML."""
    def __init__(self, data: UserInput):
        self.name = data.name
        self.statut = data.statut
        self.start = data.start
        self.end = data.end
        self.zone = data.zone
        self.interests = data.interests
        self.table = data.table

    def match_score(self, other):
        """Calcule un score de match entre 0 et 100."""

        # Convert times to datetime objects for easier comparison and duration calculation
        # Using a dummy date '2000-01-01' as only time matters for overlap
        format_str = "%H:%M"
        self_start_dt = datetime.strptime(self.start, format_str)
        self_end_dt = datetime.strptime(self.end, format_str)
        other_start_dt = datetime.strptime(other.start, format_str)
        other_end_dt = datetime.strptime(other.end, format_str)

        # Calculate overlap interval
        overlap_start = max(self_start_dt, other_start_dt)
        overlap_end = min(self_end_dt, other_end_dt)

        # If there is no overlap or overlap end time is before overlap start time
        if overlap_start >= overlap_end:
            return 0

        # Calculate overlap duration in minutes
        overlap_duration = (overlap_end - overlap_start).total_seconds() / 60

        # If overlap duration is less than 30 minutes, score is 0
        if overlap_duration < 30:
            return 0

        score = 20 # Start with 20 points for sufficient overlapping schedule

        # 1) Table similaire
        if self.table and other.table and self.table == other.table:
            score += 20

        # 2) Zone identique
        if self.zone and other.zone and self.zone == other.zone:
            score += 30

        # 3) Intérêts communs
        commun = set(self.interests) & set(other.interests)
        # Each common interest now gives 15 points, capped at a maximum of 30 points for interests
        score += min(len(commun) * 15, 30)

        return score


# ---------------------------
# BASE DE DONNÉES TEMPORAIRE
# ---------------------------

etudiants: List[Individu] = []
historique: List[Individu] = []


# ---------------------------
# ENDPOINT PRINCIPAL UTILISÉ PAR TON HTML
# ---------------------------

@app.post("/api/matches")
def trouver_match(data: UserInput):

    # Créer un nouvel étudiant
    nouvel = Individu(data)

    # Sauvegarder
    etudiants.append(nouvel)
    historique.append(nouvel)

    # Chercher des matchs
    suggestions = []

    for other in etudiants:
        if other is not nouvel:
            score = nouvel.match_score(other)

            if score > 0:  # On ne montre que si un début de match existe
                suggestions.append({
                    "name": other.name,
                    "statut": other.statut,
                    "zone": other.zone,
                    "start": other.start,
                    "end": other.end,
                    "interests": other.interests,
                    "table": other.table,
                    "matchScore": score
                })

    # Trier du meilleur match au moins bon
    suggestions.sort(key=lambda x: x["matchScore"], reverse=True)

    return {
        "matches": suggestions,
        "history": [
            {
                "name": user.name,
                "statut": user.statut,
                "table": user.table
            }
            for user in historique
        ]
    }

# Pour tester séparément nos parties de code Python:
if __name__ == "__main__":
  userinput1 = UserInput(
      name="Shivani",
      statut="Etudiant",
      start="12:00",
      end="12:50",
      zone="Resto U",
      interests=["Informatique", "Musique"],
      table="Etudiants"
  )

  userinput2 = UserInput(
      name="Emilie",
      statut="Etudiant",
      start="12:20",
      end="13:40",
      zone="Resto U",
      interests=["Danse", "Musique"],
      table="Etudiants"
  )

  userinput3 = UserInput(
      name="Nora",
      statut="Thesard",
      start="12:40",
      end="13:15",
      zone="Resto U",
      interests=["Sport", "Musique"],
      table="Profs"
  )
  etudiant1 = Individu(userinput1)
  etudiant2 = Individu(userinput2)
  etudiant3 = Individu(userinput3)
  score = etudiant1.match_score(etudiant2)
  score2 = etudiant1.match_score(etudiant3)
  score3 = etudiant2.match_score(etudiant3)
  print(score,score2,score3)


85 0 65


# FrontEnd (HTML, CSS, Json, JavaScript)

In [ ]:
<!doctype html>
<html lang="fr">
<head>
  <meta charset="utf-8"/>
  <meta name="viewport" content="width=device-width,initial-scale=1"/>
  <title>EatTogether — Trouve ton compagnon de midi</title>

  <link href="https://fonts.googleapis.com/css2?family=Inter:wght@300;400;600;700&family=Poppins:wght@600&display=swap" rel="stylesheet">

  <style>
    :root {
      --bg: #FFF8F0;
      --card: #FFFFFF;
      --primary: #E37A30;
      --accent: #F5C16C;
      --text: #4A341F;
      --muted: #8D7D6B;
      --radius: 14px;
      --glass: rgba(255, 255, 255, 0.7);
      --transition: 0.3s ease;
    }

    * { box-sizing: border-box; }
    html, body { height: 100%; margin: 0; font-family: 'Inter', sans-serif; }
    body {
      background: linear-gradient(180deg, var(--bg), #FFF2E0);
      color: var(--text);
      padding: 24px;
    }

    .container { max-width: 1000px; margin: 0 auto; }

    header {
      display: flex; justify-content: space-between; align-items: center;
      padding: 12px 0;
    }

    .brand {
      display: flex; gap: 12px; align-items: center;
    }

    .logo {
      width: 56px; height: 56px; border-radius: 12px;
      background: linear-gradient(135deg, var(--primary), var(--accent));
      color: white; display: flex; align-items: center; justify-content: center;
      font-family: 'Poppins'; font-weight: bold;
      box-shadow: 0 8px 20px rgba(227, 122, 48, 0.3);
    }

    .brand h1 {
      margin: 0; font-size: 20px; color: var(--primary);
    }

    .muted {
      font-size: 13px; color: var(--muted);
    }

    .cta {
      background: var(--primary); color: white; padding: 10px 16px;
      border: none; border-radius: var(--radius); font-weight: bold;
      cursor: pointer; transition: background var(--transition), transform var(--transition);
    }

    .cta:hover {
      background: var(--accent);
      transform: translateY(-2px);
    }

    .main {
      display: grid;
      grid-template-columns: 1fr 360px;
      gap: 24px;
      margin-top: 24px;
    }

    @media (max-width: 880px) {
      .main { grid-template-columns: 1fr; }
      .right-col { order: 2; }
    }

    .card {
      background: var(--card); border-radius: var(--radius);
      padding: 20px; box-shadow: 0 8px 20px rgba(0,0,0,0.05);
    }

    .hero-title {
      display: flex; gap: 14px; align-items: center;
    }

    .hero-title h2 {
      margin: 0; font-size: 22px; font-family: 'Poppins';
    }

    form label {
      font-size: 14px; color: var(--muted);
      margin-top: 12px; display: block;
    }

    input[type="text"], select, input[type="time"] {
      width: 100%; padding: 10px;
      border: 1px solid #E0C9B4;
      border-radius: var(--radius);
      background: white; color: var(--text);
    }

    .row {
      display: flex; gap: 12px;
    }

    .interests {
      display: flex; flex-wrap: wrap;
      gap: 10px; margin-top: 8px;
    }

    .chip {
      padding: 8px 12px; border-radius: 999px;
      border: 1px solid #E7D4BC; background: #FFF4E5;
      font-size: 13px; cursor: pointer;
      color: var(--text);
    }

    .chip.active {
      background: var(--primary); color: white;
      border: none;
    }

    .btn-primary {
      background: var(--primary); color: white;
      padding: 12px 16px;
      width: 100%;
      border: none;
      margin-top: 16px;
      border-radius: var(--radius);
      font-weight: 600;
      cursor: pointer;
    }

    .btn-primary:hover {
      background: var(--accent);
    }

    .choice-container {
      display: flex; gap: 15px; flex-wrap: wrap;
      margin-top: 12px;
    }

    .choice {
      flex: 1; min-width: 120px;
    }

    .choice input {
      display: none;
    }

    .choice label {
      display: block; padding: 18px 14px;
      border: 2px solid var(--primary);
      border-radius: 12px; text-align: center;
      background-color: #fff5e9;
      font-weight: 600; font-size: 15px;
      color: var(--text);
      cursor: pointer;
    }

    .choice input:checked + label {
      background-color: var(--primary);
      color: white;
      box-shadow: 0 4px 14px rgba(227, 122, 48, 0.3);
    }

    .people-list {
      display: flex; flex-direction: column; gap: 12px;
    }

    .person {
      display: flex; gap: 12px; align-items: center;
      padding: 12px; border-radius: var(--radius);
      background: #FFF8F0;
      border: 1px solid #EADBCB;
    }

    .avatar {
      width: 56px; height: 56px;
      border-radius: 12px;
      background: #FFE6C7;
      display: flex; align-items: center; justify-content: center;
      font-weight: bold; color: var(--primary);
    }

    .match-badge {
      background: var(--accent); color: white;
      padding: 6px 10px;
      border-radius: 999px;
      font-weight: 600; font-size: 13px;
    }

    footer {
      text-align: center; font-size: 13px; color: var(--muted);
      margin-top: 24px;
    }

    .empty {
      padding: 24px; text-align: center; color: var(--muted);
    }

    /* --- PLAN JUSSIEU --- */
    .map-block {
      margin-top: 16px;
    }

    .map-wrapper {
      position: relative;
      margin-top: 8px;
      border-radius: var(--radius);
      overflow: hidden;
      box-shadow: 0 6px 16px rgba(0, 0, 0, 0.06);
      background: #f7ede0;
    }

    .campus-map {
      width: 100%;
      display: block;
    }

    .map-pin {
      position: absolute;
      transform: translate(-50%, -50%);
      padding: 6px 10px;
      border-radius: 999px;
      border: none;
      background: var(--primary);
      color: #fff;
      font-size: 11px;
      cursor: pointer;
      white-space: nowrap;
      box-shadow: 0 4px 10px rgba(227, 122, 48, 0.3);
      transition: transform var(--transition), box-shadow var(--transition), background var(--transition);
    }

    .map-pin:hover {
      transform: translate(-50%, -50%) scale(1.05);
      background: var(--accent);
      box-shadow: 0 6px 16px rgba(227, 122, 48, 0.45);
    }
  </style>
</head>
<body>
  <div class="container">
    <header>
      <div class="brand">
        <div class="logo">ET</div>
        <div>
          <h1>EatTogether</h1>
          <div class="muted">Trouve ton compagnon de déjeuner</div>
        </div>
      </div>
      <button class="cta" id="demoBtn">Connexion</button>
    </header>

    <main class="main">
      <!-- Formulaire -->
      <section class="card">
        <div class="hero-title">
          <div style="font-size:28px">🍽️</div>
          <div>
            <h2>Manger seul, c'est fini !</h2>
            <div class="muted">Renseigne ta dispo et ta zone — on t’ajoute à la bande :)</div>
          </div>
        </div>

        <form id="matchForm">
          <label for="name">Prénom / Pseudo</label>
          <input id="name" name="name" type="text" placeholder="Ex : Alice" required>

          <label for="statut">Statut</label>
          <select id="statut" name="statut" required>
            <option value="">Choisir</option>
            <option>Etudiant</option>
            <option>Prof</option>
            <option>Thésard</option>
          </select>

          <div class="row">
            <div style="flex:1">
              <label for="start">Heure début</label>
              <input id="start" name="start" type="time" required>
            </div>
            <div style="width:140px">
              <label for="end">Heure fin</label>
              <input id="end" name="end" type="time" required>
            </div>
          </div>
          </select>

          <!-- Plan de Jussieu -->
          <div class="map-block">
            <p class="muted">Repère les lieux sur le plan de Jussieu et clique sur un point :</p>

            <div class="map-wrapper">
              <img src="plan-jussieu.jpeg" alt="Plan du campus Sorbonne Jussieu" class="campus-map">

              <button type="button" class="map-pin" style="top: 35%; left: 40%;" data-zone="Resto U">Resto U</button>
              <button type="button" class="map-pin" style="top: 50%; left: 60%;" data-zone="Cafétéria">Cafétéria</button>
              <button type="button" class="map-pin" style="top: 70%; left: 30%;" data-zone="Pelouse de la fac">Pelouse</button>
              <button type="button" class="map-pin" style="top: 20%; left: 75%;" data-zone="Maison de la vie étudiante">MVE</button>
              <button type="button" class="map-pin" style="top: 60%; left: 15%;" data-zone="Esclangon">Esclangon</button>
            </div>
          </div>

          <div style="margin-top:12px">
            <label>Centres d'intérêt </label>
            <div class="interests" role="list" id="interests">
              <div class="chip" data-val="Sport">Sport</div>
              <div class="chip" data-val="Informatique">Informatique</div>
              <div class="chip" data-val="Cuisine & Pâtisserie">Cuisine & Pâtisserie</div>
              <div class="chip" data-val="Musique">Musique</div>
              <div class="chip" data-val="Jeux vidéo">Jeux vidéo</div>
              <div class="chip" data-val="Jeux de société">Jeu de société</div>
              <div class="chip" data-val="Voyages">Voyages</div>
              <div class="chip" data-val="Engagement sociétal">Engagement sociétal</div>
              <div class="chip" data-val="Randonnée & Nature">Randonnée & Nature</div>
              <div class="chip" data-val="Lecture">Lecture</div>
              <div class="chip" data-val="Danse">Danse</div>
              <div class="chip" data-val="Sorties entre amis">Sorties entre amis</div>
              <div class="chip" data-val="Festivals & Evenements">Festivals & Evenements</div>
              <div class="chip" data-val="Art & Culture">Art & Culture</div>
            </div>
          </div>

          <div style="margin-top:20px">
            <p><strong>La table qui te ressemble :</strong></p>
            <div class="choice-container">
              <div class="choice">
                <input type="radio" id="etudiants" name="table" value="Etudiants" required>
                <label for="etudiants">🎓 Étudiants</label>
              </div>
              <div class="choice">
                <input type="radio" id="profs" name="table" value="Profs">
                <label for="profs">👩‍🏫 Profs</label>
              </div>
              <div class="choice">
                <input type="radio" id="thesard" name="table" value="Thésard">
                <label for="thesard">📚 Thésard</label>
              </div>
              <div class="choice">
                <input type="radio" id="mix" name="table" value="Mix and Chill">
                <label for="mix">🎉 Mix & Chill</label>
              </div>
            </div>
          </div>

          <button type="submit" class="btn-primary">Trouve ton crew !</button>
        </form>
      </section>

      <!-- Résultats -->
      <aside class="right-col card">
        <div class="section-title">
          <h3 style="margin:0">Suggestions près de toi</h3>
          <div class="muted">Mises à jour en temps réel</div>
        </div>

        <div id="results" class="people-list">
          <div class="empty">Aucun match pour l'instant.</div>
        </div>

        <div style="margin-top:16px">
          <h4>Historique</h4>
          <div id="history" class="people-list"></div>
        </div>
      </aside>
    </main>

    <footer>
      EatTogether • Projet étudiant — Respecte la vie privée des utilisateurs.
    </footer>
  </div>

  <script>
    // Gestion des centres d'intérêt (chips)
    document.querySelectorAll('.chip').forEach(chip => {
      chip.addEventListener('click', () => {
        chip.classList.toggle('active');
      });
    });

    // Gestion des points cliquables sur le plan
    document.querySelectorAll('.map-pin').forEach(pin => {
      pin.addEventListener('click', () => {
        const zoneSelect = document.getElementById('zone');
        const zoneValue = pin.getAttribute('data-zone');

        // On met la zone choisie dans le select
        zoneSelect.value = zoneValue;

        // Petit effet visuel
        pin.animate(
          [
            { transform: 'translate(-50%, -50%) scale(1)' },
            { transform: 'translate(-50%, -50%) scale(1.1)' },
            { transform: 'translate(-50%, -50%) scale(1)' }
          ],
          { duration: 200 }
        );
      });
    });
  </script>
</body>
</html>